In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

In [2]:
N = 100000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

[ 1.00421271 -2.01061045  0.99067306] 284106.497917
[ 1.00421271 -2.01061045  0.99067306] 200342.052255
[ 1.00421271 -2.01061045  0.99067306] 200342.052255


In [4]:
model.coef_

array([ 1.00421271, -2.01061045,  0.99067306])

## Logistic Model

In [5]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [6]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

[ 0.93148515 -1.93240319  0.96636337] -134231.342147
[ 0.98017002 -1.97756427  1.00779825] -124881.638533
[ 0.98091644 -1.97910972  1.00878297] -124830.80088
[ 0.98091663 -1.97911016  1.00878325] -124830.792043
[ 0.98091663 -1.97911016  1.00878325] -124830.792043


In [7]:
model.coef_

array([ 0.98091663, -1.97911016,  1.00878325])

## Poission Model

In [8]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [9]:
min(mu)

0.36964965986445064

In [10]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

[ 1.40534054 -0.50380147  0.26177505] 142785.205549
[ 1.10125487 -1.27011     0.65572987] -114559.800801
[ 1.0149078  -1.84865107  0.93196045] -171773.278566
[ 1.0029819 -1.9908049  0.9876839] -179605.28027
[ 1.00275734 -1.99707632  0.98955184] -179881.235171
[ 1.00275751 -1.99708798  0.98955441] -179881.707689
[ 1.00275751 -1.99708798  0.98955441] -179881.70769


In [11]:
min(model.predict(X))

0.3717377260272382